In [3]:
import joblib
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.datasets import fetch_dataset

# Directorio donde se guardarán y cargarán los modelos
save_dir = 'C:\\Users\\arzua\\OneDrive\\Escritorio\\preelec\\src\\'

# Cargar datos
datos = fetch_dataset(name='vic_electricity', raw=True)
datos['Time'] = pd.to_datetime(datos['Time'], format='%Y-%m-%dT%H:%M:%SZ')
datos = datos.set_index('Time')
datos = datos.asfreq('30min')
datos = datos.sort_index()
datos = datos.drop(columns='Date')
datos = datos.resample(rule='H', closed='left', label='right').mean()
datos = datos.loc['2012-01-01 00:00:00': '2014-12-30 23:00:00'].copy()
fin_train = '2013-12-31 23:59:00'
fin_validacion = '2014-11-30 23:59:00'
datos_train = datos.loc[:fin_train, :].copy()
datos_val = datos.loc[fin_train:fin_validacion, :].copy()
datos_test = datos.loc[fin_validacion:, :].copy()

# Crear y entrenar el modelo baseline
forecaster_baseline = ForecasterEquivalentDate(
    offset=pd.DateOffset(days=1),
    n_offsets=1
)
forecaster_baseline.fit(y=datos.loc[:fin_validacion, 'Demand'])

# Crear y entrenar el modelo autoregresivo recursivo
forecaster_autoreg = ForecasterAutoreg(
    regressor=LGBMRegressor(random_state=15926, verbose=-1),
    lags=24
)
forecaster_autoreg.fit(y=datos.loc[:fin_validacion, 'Demand'])

# Crear y entrenar el modelo autoregresivo recursivo con variables exógenas
# Para simplicidad, las variables exógenas se generan de manera arbitraria.
# En un caso real, deberías definir tus variables exógenas según el problema.
exog_features = ['Temperature', 'Holiday']
forecaster_autoreg_exog = ForecasterAutoreg(
    regressor=LGBMRegressor(random_state=15926, verbose=-1),
    lags=24
)
forecaster_autoreg_exog.fit(
    y=datos.loc[:fin_validacion, 'Demand'],
    exog=datos.loc[:fin_validacion, exog_features]
)

# Guardar los modelos entrenados
joblib.dump(forecaster_baseline, save_dir + 'forecaster_baseline.pkl')
joblib.dump(forecaster_autoreg, save_dir + 'forecaster_autoreg.pkl')
joblib.dump(forecaster_autoreg_exog, save_dir + 'forecaster_autoreg_exog.pkl')

# Cargar los modelos entrenados
forecaster_baseline_loaded = joblib.load(save_dir + 'forecaster_baseline.pkl')
forecaster_autoreg_loaded = joblib.load(save_dir + 'forecaster_autoreg.pkl')
forecaster_autoreg_exog_loaded = joblib.load(save_dir + 'forecaster_autoreg_exog.pkl')

# Comprobar que los modelos cargados funcionan correctamente
print(forecaster_baseline_loaded)
print(forecaster_autoreg_loaded)
print(forecaster_autoreg_exog_loaded)


vic_electricity
---------------
Half-hourly electricity demand for Victoria, Australia
O'Hara-Wild M, Hyndman R, Wang E, Godahewa R (2022).tsibbledata: Diverse
Datasets for 'tsibble'. https://tsibbledata.tidyverts.org/,
https://github.com/tidyverts/tsibbledata/.
https://tsibbledata.tidyverts.org/reference/vic_elec.html
Shape of the dataset: (52608, 5)


C:\Users\arzua\AppData\Local\Temp\ipykernel_1468\558530577.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  datos = datos.resample(rule='H', closed='left', label='right').mean()


ForecasterEquivalentDate 
Offset: <DateOffset: days=1> 
Number of offsets: 1 
Aggregation function: mean 
Window size: 24 
Training range: [Timestamp('2012-01-01 00:00:00'), Timestamp('2014-11-30 23:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: h 
Creation date: 2024-06-06 07:59:05 
Last fit date: 2024-06-06 07:59:05 
Skforecast version: 0.12.1 
Python version: 3.11.7 
Forecaster id: None 

ForecasterAutoreg 
Regressor: LGBMRegressor(random_state=15926, verbose=-1) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
Transformer for y: None 
Transformer for exog: None 
Window size: 24 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: [Timestamp('2012-01-01 00:00:00'), Timestamp('2014-11-30 23:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: h 
Regressor parameters: {'boosting_type': 'gbdt